# Quantitative Momentum Strategy
"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## importing Our List of Stocks
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
symbol = 'AAPL'
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN



## Making Our First API Call
It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:


symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2220632171698,
 'week52high': 140.04,
 'week52low': 56.5,
 'week52change': 0.704796305278926,
 'sharesOutstanding': 17645183196,
 'float': 0,
 'avg10Volume': 121976176,
 'avg30Volume': 119833531,
 'day200MovingAvg': 114.9,
 'day50MovingAvg': 126.9,
 'employees': 0,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.8054097494007075,
 'dividendYield': 0.006494108007528707,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-25',
 'nextEarningsDate': '0',
 'peRatio': 38.7316904147803,
 'beta': 1.1788473747388155,
 'maxChangePercent': 47.84559330876382,
 'year5ChangePercent': 4.637390033460029,
 'year2ChangePercent': 2.53673607361653,
 'year1ChangePercent': 0.7335460521174816,
 'ytdChangePercent': -0.04775426297326393,
 'month6ChangePercent': 0.3637716307927331,
 'month3ChangePercent': 0.1207427613258214,
 'month1ChangePercent': 0.03699713525448877,
 'day30ChangePercent': 0.02331977286662571,
 'day5ChangePercent': -0.05371819149382439}

## Parsing Our API Call
This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']


0.7335460521174816

## Executing A Batch API Call & Building Our DataFrame
Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

### Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.393,0.465867,N/A
1,AAL,15.971,-0.439233,N/A
2,AAP,171.699,0.0685836,N/A
3,AAPL,134.740,0.724826,N/A
4,ABBV,110.830,0.25647,N/A
...,...,...,...,...
500,YUM,107.761,0.0684889,N/A
501,ZBH,161.280,0.0952641,N/A
502,ZBRA,418.120,0.551282,N/A
503,ZION,52.370,-0.0145865,N/A


## Removing Low-Momentum Stocks
The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [7]:

final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,42.190,2.3898,N/A
1,ALB,186.680,1.51888,N/A
2,LB,47.746,1.43447,N/A
3,FCX,31.761,1.3917,N/A
4,NVDA,543.750,1.18203,N/A
5,PYPL,234.930,1.07897,N/A
6,WST,315.960,0.984106,N/A
7,ALGN,568.800,0.906681,N/A
8,AMD,96.610,0.894058,N/A
9,CDNS,143.000,0.876671,N/A


## Calculating the Number of Shares to Buy
Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:

def portfolio_input():
    global portfolio_size
    #portfolio_size = input("Enter the value of your portfolio:")
    portfolio_size=1000000
    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\ersin\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,42.190,2.3898,464
1,ALB,186.680,1.51888,105
2,LB,47.746,1.43447,410
3,FCX,31.761,1.3917,617
4,NVDA,543.750,1.18203,36
5,PYPL,234.930,1.07897,83
6,WST,315.960,0.984106,62
7,ALGN,568.800,0.906681,34
8,AMD,96.610,0.894058,202
9,CDNS,143.000,0.876671,137


# Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

-- High-quality momentum stocks show "slow and steady" outperformance over long periods of time

-- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

1-month price returns

3-month price returns

6-month price returns

1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.184,N/A,0.481731,N/A,0.395457,N/A,0.212243,N/A,0.0720484,N/A,N/A
1,AAL,16.090,N/A,-0.449372,N/A,0.217784,N/A,0.247203,N/A,-0.0557681,N/A,N/A
2,AAP,171.803,N/A,0.0693786,N/A,0.201569,N/A,0.0748268,N/A,0.095237,N/A,N/A
3,AAPL,136.572,N/A,0.722272,N/A,0.362475,N/A,0.122352,N/A,0.0372356,N/A,N/A
4,ABBV,108.550,N/A,0.252979,N/A,0.097711,N/A,0.24745,N/A,-0.0160371,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.991,N/A,0.0657886,N/A,0.215914,N/A,0.166515,N/A,0.0053972,N/A,N/A
501,ZBH,164.820,N/A,0.0923026,N/A,0.36831,N/A,0.144516,N/A,0.079992,N/A,N/A
502,ZBRA,417.970,N/A,0.556463,N/A,0.517959,N/A,0.472794,N/A,0.0438343,N/A,N/A
503,ZION,50.760,N/A,-0.0144484,N/A,0.513442,N/A,0.598537,N/A,0.104901,N/A,N/A


## Calculating Momentum Percentiles¶
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

One-Year Price Return

Six-Month Price Return

Three-Month Price Return

One-Month Price Return

Here's how we'll do this:

In [26]:
hqm_dataframe['One-Month Price Return'] = hqm_dataframe['One-Month Price Return'].astype(float)
hqm_dataframe.dtypes

Ticker                            object
Price                            float64
Number of Shares to Buy           object
One-Year Price Return            float64
One-Year Return Percentile        object
Six-Month Price Return           float64
Six-Month Return Percentile       object
Three-Month Price Return         float64
Three-Month Return Percentile     object
One-Month Price Return           float64
One-Month Return Percentile       object
HQM Score                         object
dtype: object

In [27]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

C:\Users\ersin\Anaconda3\lib\site-packages\scipy\stats\stats.py:2017: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
C:\Users\ersin\Anaconda3\lib\site-packages\scipy\stats\stats.py:2018: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)


0       0.875248
1      0.0178218
2       0.447525
3       0.956436
4       0.685149
         ...    
500     0.441584
501     0.483168
502     0.918812
503     0.306931
504     0.720792
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.726733
1      0.449505
2      0.413861
3      0.691089
4      0.241584
         ...   
500    0.441584
501     0.70297
502    0.861386
503    0.853465
504    0.453465
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.60396
1      0.653465
2       0.29901
3      0.415842
4      0.655446
         ...   
500    0.506931
501    0.445545
502     0.89505
503    0.946535
504    0.215842
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.782178
1      0.0831683
2       0.871287
3       0.582178
4       0.261386
         ...    
500     0.388119
501     0.809901
502     0.625743
503      0.89505
504     0.710891
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.184,N/A,0.481731,0.875248,0.395457,0.726733,0.212243,0.60396,0.072048,0.782178,N/A
1,AAL,16.090,N/A,-0.449372,0.0178218,0.217784,0.449505,0.247203,0.653465,-0.055768,0.0831683,N/A
2,AAP,171.803,N/A,0.069379,0.447525,0.201569,0.413861,0.074827,0.29901,0.095237,0.871287,N/A
3,AAPL,136.572,N/A,0.722272,0.956436,0.362475,0.691089,0.122352,0.415842,0.037236,0.582178,N/A
4,ABBV,108.550,N/A,0.252979,0.685149,0.097711,0.241584,0.247450,0.655446,-0.016037,0.261386,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.991,N/A,0.065789,0.441584,0.215914,0.441584,0.166515,0.506931,0.005397,0.388119,N/A
501,ZBH,164.820,N/A,0.092303,0.483168,0.368310,0.70297,0.144516,0.445545,0.079992,0.809901,N/A
502,ZBRA,417.970,N/A,0.556463,0.918812,0.517959,0.861386,0.472794,0.89505,0.043834,0.625743,N/A
503,ZION,50.760,N/A,-0.014448,0.306931,0.513442,0.853465,0.598537,0.946535,0.104901,0.89505,N/A


## Calculating the HQM Score
We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module

In [29]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.184,N/A,0.481731,0.875248,0.395457,0.726733,0.212243,0.60396,0.072048,0.782178,0.74703
1,AAL,16.090,N/A,-0.449372,0.0178218,0.217784,0.449505,0.247203,0.653465,-0.055768,0.0831683,0.30099
2,AAP,171.803,N/A,0.069379,0.447525,0.201569,0.413861,0.074827,0.29901,0.095237,0.871287,0.507921
3,AAPL,136.572,N/A,0.722272,0.956436,0.362475,0.691089,0.122352,0.415842,0.037236,0.582178,0.661386
4,ABBV,108.550,N/A,0.252979,0.685149,0.097711,0.241584,0.247450,0.655446,-0.016037,0.261386,0.460891
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.991,N/A,0.065789,0.441584,0.215914,0.441584,0.166515,0.506931,0.005397,0.388119,0.444554
501,ZBH,164.820,N/A,0.092303,0.483168,0.368310,0.70297,0.144516,0.445545,0.079992,0.809901,0.610396
502,ZBRA,417.970,N/A,0.556463,0.918812,0.517959,0.861386,0.472794,0.89505,0.043834,0.625743,0.825248
503,ZION,50.760,N/A,-0.014448,0.306931,0.513442,0.853465,0.598537,0.946535,0.104901,0.89505,0.750495


## Selecting the 50 Best Momentum Stocks
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [30]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [31]:
portfolio_input()

In [32]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\ersin\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.184,150,0.481731,0.875248,0.395457,0.726733,0.212243,0.60396,0.072048,0.782178,0.74703
1,AAL,16.090,1218,-0.449372,0.0178218,0.217784,0.449505,0.247203,0.653465,-0.055768,0.0831683,0.30099
2,AAP,171.803,114,0.069379,0.447525,0.201569,0.413861,0.074827,0.29901,0.095237,0.871287,0.507921
3,AAPL,136.572,143,0.722272,0.956436,0.362475,0.691089,0.122352,0.415842,0.037236,0.582178,0.661386
4,ABBV,108.550,180,0.252979,0.685149,0.097711,0.241584,0.247450,0.655446,-0.016037,0.261386,0.460891
5,ABC,113.760,172,0.279538,0.736634,0.064122,0.184158,0.119134,0.407921,0.032429,0.558416,0.471782
6,ABMD,333.510,58,0.809676,0.966337,0.208911,0.429703,0.220357,0.615842,0.182408,0.972277,0.74604
7,ABT,116.010,169,0.301866,0.772277,0.200312,0.411881,0.042267,0.223762,0.022240,0.508911,0.479208
8,ACN,270.490,72,0.269420,0.708911,0.213388,0.437624,0.194481,0.558416,0.029421,0.546535,0.562871
9,ADBE,480.260,40,0.411022,0.851485,0.031741,0.146535,-0.027622,0.10099,-0.041470,0.126733,0.306436


## Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [33]:

writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

##  Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

String format for tickers

\$XX.XX format for stock prices

\$XX,XXX format for market capitalization

Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [34]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [35]:
writer.save()